# Naive Bayes

V tomto notebooku se budeme zabývat využitím klasifikace pomocí Naivního Bayese. Speciálně se budeme soustředit na klasifikaci textů.
 
Základem pro tento dokument je tutorial ze scikit-learn zaměřený na analýzu textů [zde](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html).

In [7]:
import pandas as pd
import numpy as np
from scipy.special import logsumexp
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.)

ModuleNotFoundError: No module named 'pandas'

## Načtení dat

Využijeme data ze zdroje [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), který obsahuje různě kategorizované texty z internetových diskusí.

Pro jednoduchost se zaměříme pouze na dvě kategorie - hokej a auta.

In [3]:
categories = ['rec.sport.hockey', 'rec.autos']
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=1)
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=1)

In [4]:
# Prozkoumání trénovacích dat
print('Kategorie:', train_data.target_names)
print('Train data length:', len(train_data.data))

# print(train_data.data[0])
print('Kategorie prvního dokumentu:',train_data.target_names[train_data.target[0]])

Kategorie: ['rec.autos', 'rec.sport.hockey']
Train data length: 1194
Kategorie prvního dokumentu: rec.autos


## Transformace do bag-of-words reprezentace

Použijeme k tomu CountVectorizer ze scikit-learn.

In [5]:
# Nejprve načteme slovník
with open('vocabulary.txt','r') as f:
    vocab=f.read().splitlines()
print(len(vocab))

61188


In [6]:
count_vect = CountVectorizer(vocabulary = vocab)
X_train_counts = count_vect.fit_transform(train_data.data)
print('Bag of words shape', X_train_counts.shape)
print('Bag of words type', type(X_train_counts))

Bag of words shape (1194, 61188)
Bag of words type <class 'scipy.sparse.csr.csr_matrix'>


Výstupem je scipy.sparse matrix.

In [7]:
# zobrazení prvního řádku - tj.příznaků prvního dokumentu
X_train_counts[0,:20].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 2, 0, 0, 0, 0]],
      dtype=int64)

We can also extract the vocabulary...

In [8]:
# several words from the dictionary together with their indices in the dictionary
print(type(count_vect.vocabulary_))
print(len(count_vect.vocabulary_))
print(len(vocab))

print({vocab[i]:X_train_counts[0,i] for i in range(20)})

<class 'dict'>
61188
61188
{'archive': 0, 'name': 0, 'atheism': 0, 'resources': 0, 'alt': 0, 'last': 0, 'modified': 0, 'december': 0, 'version': 0, 'atheist': 0, 'addresses': 0, 'of': 6, 'organizations': 0, 'usa': 0, 'freedom': 0, 'from': 2, 'religion': 0, 'foundation': 0, 'darwin': 0, 'fish': 0}


### TASK 1 - aplikujte nejprve jednoduchý model - Bernoulli Naive Bayes

* Reprezentujte dokument pomocí indikátorů výskytů slov ze slovníku vocab
* Natrénujte Naivního Bayese s Bernoulliho rozdělením příznaků
* Otestujte kvalitu predikce na ručně určených dokumentech
* Odhadněte přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [9]:
# Vlastní dokumenty pro testování
docs_new = ["Lets play hockey.", "I don't like their seats"]

# alternatives to play with
# docs_new = ["Lets play hockey.", "I don't like their game"]

In [10]:
# Your code here

# get indicators
ind_vect = CountVectorizer(binary = True, vocabulary = vocab)
X_train_indicators = ind_vect.fit_transform(train_data.data)

# Train the classifier
clfi = BernoulliNB().fit(X_train_indicators, train_data.target)

# výpočet indikátorů pomocí natrénovaného CountVectorizer v objektu ind_vect
X_new_indicators = ind_vect.transform(docs_new)

# predikce
predicted = clfi.predict(X_new_indicators)
probabs = clfi.predict_proba(X_new_indicators)

# print results
for doc, category, probab in zip(docs_new, predicted, probabs):
    print('%r => %s' % (doc, train_data.target_names[category]))
    print(probab)

# Odhad accuracy pomocí trénovací množiny

# výpočet indikátorů pomocí natrénovaného CountVectorizer v objektu ind_vect
X_new_indicators = ind_vect.transform(test_data.data)

# predikce
predicted = clfi.predict(X_new_indicators)

print(np.mean(predicted == test_data.target))
print(clfi.score(X_new_indicators, test_data.target))

'Lets play hockey.' => rec.sport.hockey
[0.13849 0.86151]
"I don't like their seats" => rec.autos
[0.99945 0.00055]
0.9911949685534591
0.9911949685534591


### TASK 2 - aplikujme složitější model - Multinomial Naive Bayes

* Reprezentujte dokument pomocí počtů výskytů slov ze slovníku vocab - tj. bag-of-words reprezentace
* Natrénujte multinomického naivního Bayese
* Otestujte kvalitu predikce na ručně určených dokumentech
* Odhadněte přesnost (acccuracy) predikce s využitím testovací množiny test_data

In [11]:
# Your code here

# get counts
count_vect = CountVectorizer(vocabulary = vocab)
X_train_counts = count_vect.fit_transform(train_data.data)

# Train the classifier
clf = MultinomialNB().fit(X_train_counts, train_data.target)

# výpočet četností pomocí natrénovaného CountVectorizer v objektu count_vect
X_new_counts = count_vect.transform(docs_new)

# predikce
predicted = clf.predict(X_new_counts)
probabs = clf.predict_proba(X_new_counts)

# print results
for doc, category, probab in zip(docs_new, predicted, probabs):
    print('%r => %s' % (doc, train_data.target_names[category]))
    print(probab)
    
# Odhad accuracy pomocí trénovací množiny

# výpočet četností pomocí natrénovaného CountVectorizer v objektu ind_vect
X_new_counts = count_vect.transform(test_data.data)

# predikce
predicted = clf.predict(X_new_counts)

print(np.mean(predicted == test_data.target))
print(clf.score(X_new_counts, test_data.target))

'Lets play hockey.' => rec.sport.hockey
[0.00006 0.99994]
"I don't like their seats" => rec.autos
[0.63915 0.36085]
0.9962264150943396
0.9962264150943396


### Task 3 - Implementujte Naivní Bayesův klasifikátor v situaci, kdy má část příznaků kategorické a část Bernoulliho rozdělení

* První příznak je kategorický (se třemi kategoriemi)
* Zbývajících 10 příznaků má Bernoulliho rozdělení

**Hint** - kategorický příznak převeďte na 3 indikátorové příznaky a odděleně použijte MultinomialNB.
Potom zvlášt odhadněte zbylé Bernoulliho příznaky a na závěr získané pravděpodobnosti pronásobte. 
Pozor - je třeba v jednom z modelů zafixovat rozdělení $Y$ na rovnoměrné - aby se pravděpodobnosti $P(Y = y)$ nenásobili dvakrát.

In [12]:
# Vytvoření datasetu
class_count = 10
X00 = np.random.choice(3, size = (class_count,1), p = [0.4,0.4,0.2])
X01 = np.random.choice(3, size = (class_count,1), p = [0.2,0.5,0.3])
X0 = np.concatenate([X00,X01])
print(X0.shape)

X10 = np.random.choice(2, size = (class_count,5), p = [0.4,0.6])
X11 = np.random.choice(2, size = (class_count,5), p = [0.6,0.4])
X1 = np.concatenate([X10,X11])
print(X1.shape)

X20 = np.random.choice(2, size = (class_count,5), p = [0.4,0.6])
X21 = np.random.choice(2, size = (class_count,5), p = [0.2,0.8])
X2 = np.concatenate([X20,X21])
print(X2.shape)

X = np.concatenate([X0,X1,X2],axis = 1)
print(X.shape)

Y = np.concatenate([np.ones(class_count-3), np.zeros(class_count+3)])
print(Y)

(20, 1)
(20, 5)
(20, 5)
(20, 11)
[1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
# Your code here

from sklearn.preprocessing import OneHotEncoder
# train multinomial
Xo = OneHotEncoder(sparse=False, categories='auto').fit_transform(X0)
# Thanks to this we dont have the different priors for Y so we don't need to subtract them
clfo = MultinomialNB(fit_prior=False).fit(Xo, Y)
# train Bernoulli
Xr =  np.concatenate([X1,X2],axis = 1)
clfr = BernoulliNB(fit_prior=True).fit(Xr,Y)

# take it together
probabs = np.exp(clfo.predict_log_proba(Xo) + clfr.predict_log_proba(Xr))
print(np.argmax(probabs, axis = 1))




# Second possibility - with both priors - gives the same result
clfo = MultinomialNB(fit_prior=True).fit(Xo, Y)
# train Bernoulli
clfr = BernoulliNB(fit_prior=True).fit(Xr,Y)
# take it together
probabs = np.exp(clfo.predict_log_proba(Xo) + clfr.predict_log_proba(Xr) - clfo.class_log_prior_)

[0 1 1 0 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0]
